In [ ]:
%Please change the path to the downloaded folder
%We run a MATLAB kernel on Jupyter. You can also alternaxtively run the individual matlab files in the repository.
cd Downloads/Speech-Enhancement-main


# Method 1:Frame-Wise PSD Estimation using Bartlett Method

## First using an Energy-based detector 

In [6]:
%Method 1:Frame-Wise PSD Estimation using Bartlett Method
%This function simply calls clean_speech.m where the code for the method is actually written.
%Please refer to the next code block where we display the code for clean_speech.m

[original,sampleRate]=audioread('clean_speech.wav');
stoi_noise=zeros(1,1);
stoi_clean=zeros(1,1);
snr_seg_noise=zeros(1,1);
snr_seg_clean=zeros(1,1);
snr_noise=zeros(1,1);
snr_clean=zeros(1,1);
noisy_signals=zeros(577920,1);
clean_signals=zeros(577920,1);

noise_levels=[-10,-5,0,5,10,15,20];


for nl=1:7

aud1=original;
noise_aud1=awgn(aud1,noise_levels(nl),'measured');


frame_size=  20*10^-3*16000;
padding_need=frame_size-mod(length(noise_aud1),frame_size);
noise_aud1=padarray(noise_aud1,[padding_need,0],0,'post');
aud1=padarray(aud1,[padding_need,0],0,'post');

threshold=findthreshold(noise_aud1,frame_size);
[clean_signalf,clean_signal_fft,acc_H_test]= clean_speech(noise_aud1,frame_size,threshold);


stoi_noise_current= stoi(aud1,noise_aud1,16000);
stoi_clean_current = stoi(aud1,clean_signalf,16000);

[snr_seg_noise_current,snr_noise_current]=v_snrseg(noise_aud1,aud1,16000);
[snr_seg_clean_current,snr_clean_current]= v_snrseg(clean_signalf,aud1,16000);

stoi_noise=horzcat(stoi_noise,stoi_noise_current);
stoi_clean=horzcat(stoi_clean,stoi_clean_current);

snr_seg_noise=horzcat(snr_seg_noise,snr_seg_noise_current);
snr_seg_clean=horzcat(snr_seg_clean,snr_seg_clean_current);

snr_noise=horzcat(snr_noise,snr_noise_current);
snr_clean=horzcat(snr_clean,snr_clean_current);

noisy_signals=horzcat(noisy_signals,noise_aud1);
clean_signals=horzcat(clean_signals,clean_signalf);


end

disp("snr before filtering")
disp(snr_noise(2:end))

disp("snr after filtering")
disp(snr_clean(2:end))


disp("snr segmental before filtering")
disp(snr_seg_noise(2:end))


disp("snr segmental after filtering")
disp(snr_seg_clean(2:end))


disp("stoi before filtering")
disp(stoi_noise(2:end))


disp("stoi after filtering")
disp(stoi_clean(2:end))

snr before filtering
   -6.2423   -1.3332    3.4497    8.1534   12.7204   17.3342   21.9495

snr after filtering
   -3.2355    0.6771    4.4422    9.4724   12.9507   17.5984   22.0033

snr segmental before filtering
  -17.8205  -12.9034   -8.0976   -3.3531    1.2561    5.8916   10.5043

snr segmental after filtering
  -13.6997   -9.9109   -6.4067   -0.5730    1.8029    7.6481   11.0853

stoi before filtering
    0.4613    0.5453    0.6484    0.7506    0.8422    0.9113    0.9541

stoi after filtering
    0.4023    0.5246    0.6353    0.7246    0.8338    0.8866    0.9484



The code for clean_speech.m is given here. This function, saved as 'clean_speech.m' in the directory, is called by the previous code block. We show the code here-


function [clean_signalf,clean_signal_fft,acc_H_test] = clean_speech(noisy_signal,frame_size,threshold)
%with multiplication of the filter in the frequency domain.
subframe_size=frame_size/10;

acc_H_test=zeros(frame_size,1);
num_frames= int32(length(noisy_signal)/frame_size);
i= [1:frame_size];
isf=[1:subframe_size];
clean_signal= zeros(1);
clean_signal_fft=zeros(1);
for n = 0:num_frames-1
    
    frame=noisy_signal(i+double(n*frame_size));

    subframe_energy=zeros(1,10);
    
    subframe_identity=zeros(1,10);
    for j = 1:10
      subframe=frame(isf+double(j*10));
      subframe_energy(j)= sum(abs(subframe).^2);
      if(subframe_energy(j)>threshold)
        subframe_identity(j)=1;
      end

    end

    n_noise=0;
    n_fft=zeros(frame_size,1);
    n_sn=0;
    sn_fft=zeros(frame_size,1);
    
    for j=1:10
        subframe=frame(isf+double(j*10));
        if(subframe_identity(j)==1)
            n_sn=n_sn+1;
            sn_fft=horzcat(sn_fft,abs(fft(subframe,frame_size)).^2);
        else
           
            n_noise=n_noise+1;
            n_fft=horzcat(n_fft,abs(fft(subframe,frame_size)).^2);

        end
          
     end
     
    if(n_noise~=0)

    avg_n_per=sum(n_fft,2)/(n_noise*subframe_size);

    else
    avg_n_per=zeros(frame_size,1);
    end
  
    if(n_sn~=0)

    avg_sn_per=sum(sn_fft,2)/(n_sn*subframe_size);
    else
     avg_sn_per=zeros(frame_size,1);
    end 
    n_power=sum(abs(avg_n_per).^2); % check for normalization by 2pi
    n_sn_power=sum(abs(avg_sn_per).^2) ;
    %designing the wiener filter using the two power spectral densities
    if (avg_sn_per==0)
        avg_sn_per=avg_n_per;
    end
   
    psd_speech= avg_sn_per-avg_n_per;
     
    for x = 1:length(psd_speech)
    if psd_speech(x) < 0
      psd_speech(x)=0;
    end
    end

    psd_noise=avg_n_per;
     
    H = psd_speech./(psd_speech+psd_noise);
 
    
   
    acc_H_test=horzcat(acc_H_test,H);
    frame_fft= fft(frame);
    clean_frame_fft= H.* frame_fft;
    clean_signal_fft=vertcat(clean_signal_fft,clean_frame_fft);
    clean_frame = ifft(clean_frame_fft);
    clean_signal=vertcat(clean_signal,clean_frame);
    


end

clean_signalf=clean_signal(2:end);

end



## Using a ZeroCrossing Based Detector

In [36]:



seed = 1;
length_frame = 0.02;
length_subframe = 0.002;
fs=16000;

[original,sampleRate]=audioread('clean_speech.wav');
stoi_noise=zeros(1,1);
stoi_clean=zeros(1,1);
snr_seg_noise=zeros(1,1);
snr_seg_clean=zeros(1,1);
snr_noise=zeros(1,1);
snr_clean=zeros(1,1);
noisy_signals=zeros(577920,1);
clean_signals=zeros(577920,1);

noise_levels=[-10,-5,0,5,10,15,20];


for nl=1:7

aud1=original';


%Read audio
clean_speech = original';

%Paddle
clean_speech_padding = [clean_speech,zeros(1,265)];
N = length(clean_speech_padding);

%Find speech start point index
for i = 1:N
    if (clean_speech_padding(i) == 0)&&(clean_speech_padding(i+1)~=0)
        start_index = i+1;
        break
    end
end

%Add noise
%Generate noise
%White Guassian noise
noise_aud1=awgn(clean_speech_padding,noise_levels(nl),'measured');
contaminated_speech=noise_aud1;
L_frame = length_frame*fs;
L_subframe = length_subframe*fs;

K_frame = N/L_frame;
K_subframe = L_frame/L_subframe;

%Segment
frames = zeros(K_frame,L_frame);
subframes = zeros(K_subframe,L_subframe,K_frame);
spc_frames = zeros(K_frame,L_frame);

for i = 1:K_frame
    frames(i,:) = contaminated_speech((i-1)*L_frame+1:i*L_frame);
    tmp = abs(fft(frames(i,:)));
    spc_frames(i,:) = tmp;
    %spc_frames(i,:) = tmp(1:round(length(tmp)/2));
    for j = 1:K_subframe
        subframes(j,:,i) = frames(i,(j-1)*L_subframe+1:j*L_subframe);
    end
end

%Calculate zero-crossings
cnt_zeros = zeros(K_frame,K_subframe);

power_noise = zeros(K_frame,L_subframe);
power_contaminated_speech = zeros(K_frame,L_subframe);
for i = 1:K_frame
    for j = 1:K_subframe
        cnt_zeros(i,j) = zero_crossings(subframes(j,:,i))/L_subframe;
    end
    zero_noise = mean(mean(cnt_zeros(1:(start_index-1)/L_frame,:)));
    thr = zero_noise; %Average value of noise and contaminated speech.
    
    index_noise = find(cnt_zeros(i,:)>thr);
    index_contaminated_speech = find(cnt_zeros(i,:)<thr);

    len_noise = length(index_noise)*L_subframe;
    len_conta = length(index_contaminated_speech)*L_subframe;

    for j = 1:K_subframe
        spc = abs(fft(subframes(j,:,i)));
        %spc = spc(1:round(length(spc)/2));
        if cnt_zeros(i,j)>thr
            power_noise(i,:) = power_noise(i,:) + abs(spc).^2/len_noise;
        else
            power_contaminated_speech(i,:) = power_contaminated_speech(i,:) + abs(spc).^2/len_conta;
        end
    end
    
end

%Estimate the PSD of clean speech
power_clean_speech = power_contaminated_speech - power_noise;
power_clean_speech(power_clean_speech<0)=0;

winner = power_clean_speech./(power_noise + power_clean_speech);

spc_clean_speech_subframe = zeros(K_subframe,L_subframe,K_frame);
est_clean_speech = 0;
for i = 1:K_frame
    est_clean_speech_frames = conv(abs(ifft(winner(i))),frames(i,:));
    est_clean_speech = [est_clean_speech,est_clean_speech_frames];
end

% for i = 1:K_frame
%     for j = 1:K_subframe
%         spc_clean_speech_subframe(j,:,i) = winner(i).*subframes(j,:,i);
%         est_clean_speech = [est_clean_speech,ifft(spc_clean_speech_subframe(j,:,i))];
%     end
% end

est_clean_speech = real(est_clean_speech);
spc_est_clean_speech = fftshift(fft(est_clean_speech));
spc_clearn_speech_padding = fftshift(fft(clean_speech_padding));

aud1=clean_speech_padding';
noise_aud1=noise_aud1';
est_clean_speech=est_clean_speech';
est_clean_speech=est_clean_speech(2:end);

stoi_noise_current= stoi(aud1,noise_aud1,16000);
stoi_clean_current = stoi(aud1,est_clean_speech,16000);

[snr_seg_noise_current,snr_noise_current]=v_snrseg(noise_aud1,aud1,16000);
[snr_seg_clean_current,snr_clean_current]= v_snrseg(est_clean_speech,aud1,16000);

stoi_noise=horzcat(stoi_noise,stoi_noise_current);
stoi_clean=horzcat(stoi_clean,stoi_clean_current);

snr_seg_noise=horzcat(snr_seg_noise,snr_seg_noise_current);
snr_seg_clean=horzcat(snr_seg_clean,snr_seg_clean_current);

snr_noise=horzcat(snr_noise,snr_noise_current);
snr_clean=horzcat(snr_clean,snr_clean_current);

noisy_signals=horzcat(noisy_signals,noise_aud1);
clean_signals=horzcat(clean_signals,est_clean_speech);


end



disp("snr before filtering")
disp(snr_noise(2:end))

disp("snr after filtering")
disp(snr_clean(2:end))


disp("snr segmental before filtering")
disp(snr_seg_noise(2:end))


disp("snr segmental after filtering")
disp(snr_seg_clean(2:end))


disp("stoi before filtering")
disp(stoi_noise(2:end))


disp("stoi after filtering")
disp(stoi_clean(2:end))




snr before filtering
   -6.2421   -1.3447    3.4612    8.1455   12.7524   17.3338   21.9738

snr after filtering
   -1.7241    1.6776    5.1066    8.0911   10.1560   11.3137   11.9205

snr segmental before filtering
  -17.8200  -12.9160   -8.0843   -3.3663    1.2849    5.8877   10.5338

snr segmental after filtering
   -7.9493   -5.4487   -2.7105    0.5193    3.2727    6.6952   10.2639

stoi before filtering
    0.4558    0.5432    0.6484    0.7526    0.8424    0.9131    0.9551

stoi after filtering
    0.2985    0.4245    0.5595    0.6827    0.7788    0.8520    0.9035



# Method 2: PSD Estimation by Welch Method, and using a constant NPSD

In [9]:
%an alternate approach- 
%first detect the noise PSD using the welch from the first 1000 frames where we know that
%there is no speech.
%let that psd be constant and then split the signal into frames.
%now, for each frame, consider the entire signal as S+N and compute its
%PSD from the entire frame. Try and use the welch method.
%Filter each frame with the wiener filter but the only difference now
%is that the noise statistics are not updated.


%advantage is that we now do not have the thresholding problem.
%disadvantage is that it wont work with changing noise 
%statistics.



[original,sampleRate]=audioread('clean_speech.wav');
stoi_noise=zeros(1,1);
stoi_clean=zeros(1,1);
snr_seg_noise=zeros(1,1);
snr_seg_clean=zeros(1,1);
snr_noise=zeros(1,1);
snr_clean=zeros(1,1);
noisy_signals=zeros(577664,1);
clean_signals=zeros(577664,1);

noise_levels=[-10,-5,0,5,10,15,20];


for nl=1:7

aud1=original;
noise_aud1=awgn(aud1,noise_levels(nl),'measured');

frame_size=128;
padding_need=frame_size-mod(length(noise_aud1),frame_size);
noise_aud1=padarray(noise_aud1,[padding_need,0],0,'post');
aud1=padarray(aud1,[padding_need,0],0,'post');

length(noise_aud1);
noise=noise_aud1(1:1024);


% Calculate the noise psd-
          
n_psd_welch=pwelch(noise);

avg_n_per=n_psd_welch(2:end);


subframe_size=frame_size/10;

acc_H_test=zeros(frame_size,1);
num_frames= int32(length(noise_aud1)/frame_size)-1;
i= [1:frame_size];
clean_signal= zeros(1);
clean_signal_fft=zeros(1);
test=zeros(128,1);
for n = 0:num_frames
    
    frame=noise_aud1(i+double(n*frame_size));

  try
    
    sn_psd_welch=pwelch(frame);
    avg_sn_per=sn_psd_welch(2:end);
  
  catch exception
      disp(exception)
      disp(n)
      disp(frame)

      break
  end
   
    psd_speech= avg_sn_per-avg_n_per;
    %check negative points and make it 0.
    for x = 1:length(psd_speech)
    if psd_speech(x) < 0
      psd_speech(x)=0;
    end
    end

    psd_noise=avg_n_per;
     
    H = psd_speech./(psd_speech+psd_noise);
    
    
    acc_H_test=horzcat(acc_H_test,H);
    frame_fft= fft(frame);
    clean_frame_fft= H.* frame_fft;
    clean_signal_fft=vertcat(clean_signal_fft,clean_frame_fft);
    clean_frame = real(ifft(clean_frame_fft));
    test=horzcat(test,clean_frame);
    clean_signal=vertcat(clean_signal,clean_frame);

end


clean_signalf2=clean_signal(2:end);

stoi_noise_current= stoi(aud1,noise_aud1,16000);
stoi_clean_current = stoi(aud1,clean_signalf2,16000);

[snr_seg_noise_current,snr_noise_current]=v_snrseg(noise_aud1,aud1,16000);
[snr_seg_clean_current,snr_clean_current]= v_snrseg(clean_signalf2,aud1,16000);

stoi_noise=horzcat(stoi_noise,stoi_noise_current);
stoi_clean=horzcat(stoi_clean,stoi_clean_current);

snr_seg_noise=horzcat(snr_seg_noise,snr_seg_noise_current);
snr_seg_clean=horzcat(snr_seg_clean,snr_seg_clean_current);

snr_noise=horzcat(snr_noise,snr_noise_current);
snr_clean=horzcat(snr_clean,snr_clean_current);

noisy_signals=horzcat(noisy_signals,noise_aud1);
clean_signals=horzcat(clean_signals,clean_signalf2);


end

disp("snr before filtering")
disp(snr_noise(2:end))

disp("snr after filtering")
disp(snr_clean(2:end))


disp("snr segmental before filtering")
disp(snr_seg_noise(2:end))


disp("snr segmental after filtering")
disp(snr_seg_clean(2:end))


disp("stoi before filtering")
disp(stoi_noise(2:end))


disp("stoi after filtering")
disp(stoi_clean(2:end))



snr before filtering
   -6.2716   -1.3428    3.4621    8.1414   12.7418   17.3600   21.9563

snr after filtering
    1.4320    3.4166    4.9166    6.1946    6.6473    8.1167    8.4643

snr segmental before filtering
  -17.8488  -12.9113   -8.0845   -3.3685    1.2727    5.9164   10.5160

snr segmental after filtering
   -8.2620   -5.1392   -2.6589   -0.4916    1.1522    3.6306    5.5818

stoi before filtering
    0.4541    0.5441    0.6462    0.7486    0.8451    0.9154    0.9532

stoi after filtering
    0.4613    0.5497    0.6501    0.7337    0.8119    0.8683    0.8949



# Method 3:Adaptive Wiener Filtering:

This method disconnects automatically from the matlab kernel because it takes a longer time to execute. Thus we show the method on only one noise level(-10db). Please execute the file AdaptiveWiener.m directly in MATLAB for the full result including all noise levels.

In [3]:
%Adaptive Wiener Filtering

[original,sampleRate]=audioread('clean_speech.wav');
%all accumulators-
stoi_noise=zeros(1,1);
stoi_clean=zeros(1,1);
snr_seg_noise=zeros(1,1);
snr_seg_clean=zeros(1,1);
snr_noise=zeros(1,1);
snr_clean=zeros(1,1);
noisy_signals=zeros(577655,1);
clean_signals=zeros(577655,1);


noise_levels=[-10,-5,0,5,10,15,20];
%add noise of different levels

for nl=1

aud1=original;
noise_aud1=awgn(aud1,noise_levels(nl),'measured');



noise=noise_aud1(1:1024);

%Steps according to Algorithm 3 in the report-
% Calculate the noise psd-
window=blackman(50);
n_psd_welch=pwelch(noise,window);
%estimate it from the first 1024 samples
var_v=mean(n_psd_welch);
%calculate the variance of the noise
M=150;
clean_signal=zeros(1,1);

for i=1024:length(noise_aud1)-M
   m_s= sum(noise_aud1(i-M:i+M))/(2*M+1);
   m_x=m_s;
   var_x=   sum((noise_aud1(i-M:i+M)-m_x).^2)/(1+2*M);

   if(var_x>var_v)
       var_s=var_x-var_v;
   else
       var_s=0;
   end

   cleaned=m_s + (var_s/(var_s+var_v)).*(noise_aud1(i)-m_s);
   clean_signal=vertcat(clean_signal,cleaned);



end
clean_signalf=clean_signal(2:end);

full_signal=vertcat(noise_aud1(1:1023),clean_signalf,noise_aud1(length(noise_aud1)-M+1:end));


stoi_noise_current= stoi(aud1,noise_aud1,16000);
stoi_clean_current = stoi(aud1,full_signal,16000);

[snr_seg_noise_current,snr_noise_current]=v_snrseg(noise_aud1,aud1,16000);
[snr_seg_clean_current,snr_clean_current]= v_snrseg(full_signal,aud1,16000);

stoi_noise=horzcat(stoi_noise,stoi_noise_current);
stoi_clean=horzcat(stoi_clean,stoi_clean_current);

snr_seg_noise=horzcat(snr_seg_noise,snr_seg_noise_current);
snr_seg_clean=horzcat(snr_seg_clean,snr_seg_clean_current);

snr_noise=horzcat(snr_noise,snr_noise_current);
snr_clean=horzcat(snr_clean,snr_clean_current);

noisy_signals=horzcat(noisy_signals,noise_aud1);
clean_signals=horzcat(clean_signals,full_signal);


end

disp("snr before filtering")
disp(snr_noise(2:end))

disp("snr after filtering")
disp(snr_clean(2:end))


disp("snr segmental before filtering")
disp(snr_seg_noise(2:end))


disp("snr segmental after filtering")
disp(snr_seg_clean(2:end))


disp("stoi before filtering")
disp(stoi_noise(2:end))


disp("stoi after filtering")
disp(stoi_clean(2:end))

snr before filtering
   -6.2271

snr after filtering
   -3.2837

snr segmental before filtering
  -17.8047

snr segmental after filtering
  -14.7778

stoi before filtering
    0.4557

stoi after filtering
    0.4760

